# Tutorial: Cluster monomeric protein conformations


### Objectives:
 1) Background
 2) Cluster example UniProt (P46060) into predicted conformations
 3) Interpret results
 4) Perform conformational state clustering on provided benchmark dataset
 5) Benchmark data results

## 1) Background and scope

For a detailed explanation of the algorithm and linear algebra used to cluster peptide chains based on global backbone C-alpha position, refer to the supplementation information here (link to bioRx). In outline, the package aims to cluster the set of parsed peptide chains based on the global conformational change difference. 

The aim of this notebook is to outline basic use and potentially applications of the `protein-cluster-conformers` package within your own Python applicaitons. The package has it's own CLI interface, which is documented in the `README.md`. 



---

## 2) Installing and importing package

### Using `pip`

You can install the latest stable version of the package from the PDBe's PyPi page using:

> `pip install protein-cluster-conformers`

### From source 

The module also can be installed from GitHub using:

> `git clone https://github.com/PDBeurope/protein-cluster-conformers.git`
> 
> `cd protein-cluster-conformers`
> 
> `pip install .`

If you would like to make edits to the source code and have those changes immediately update, replace the final line with:

> `pip install -e .`

The package is written for >=Python3.10. 

Once installed, import the package into your code with:

```python
import cluster_conformers

```

`cluster_conformers` contains many useful tools to help you with downloading, clustering and analysing your results. The most important object for clustering can be imported via:

```python
import cluster_conformers.cluster_monomers import ClusterConformations
```

---

## 3) Example: Ran GTPase-activating protein 1 (UniProt: P46060)

A detailed look at using the scripts from the command line can be found in the repository's `README.md` file. Detailed here are instructions and suggestions for implementing the clustering objects in your own Python code. 

The `ClusterConformations()` class can be used to handle the following tasks for clustering protein chains:
 - Creating pairwise CA distance matrices
 - Generate CA distance difference matrices between all chains
 - Scores global conformational changes based on these matrices
 - Cluster chains, via UPGMA clustering
 - Render dendrogram of clustering results and distance-difference maps per chain-chain comparison.

Calling the class creates an object that can perform each of the options above via a set of method calls. 

NB: _Chains are treated as single subunits, regardless of whether they occur as biological or crystallographic assemblies._ 

#### 3.1) Download structures in updated mmCIF format

To begin clustering, identify all chains which overlap the same stretch of a given UniProt sequence. Functionally, there is no limit to the amount of sequence overlap (higher is preferred) because the score function will handle this via its prefactor. However, structures with extremely little sequence overlap will return small scores, potentially getting clustered along with chains that truly share structural similarity. Gaps in the sequence are ignored when calculating chain-chain distance difference matrices but are accounted for in the score's pre-factor. 

Once all desired chains have been identified, you can quickly download them using the `fetch_updated_mmcif()` function. In the case of P46060, the code block below downloads all structures pertaining to the protein's C-terminal domain. The `fetch_updated_mmCIF()` function saves the structure files to your parsed directory, in the updated mmCIF file format. This format is especially important for clustering as it contains UniProt residue numbering from SIFTS, an essential index for matching residues between structures without performing an expensive sequence alignment. 

In [ ]:
from pathlib import Path

from cluster_conformers.utils.download_utils import fetch_updated_mmcif

# Save location -- change to yours
path_save_ummcifs = Path(
    "updated_mmcifs",
    "P46060"
)

# All structures that overlap the same stretch of P46060
pdb_structures = [
    "5d2m",
    "5d2m",
    "2grn",
    "2grp",
    "2grq",
    "2gro",
    "2grr",
    "3uip",
    "1z5s",
    "3uin",
    "2iy0",
    "3uio",
    "2io3",
    "2io2"
]

# Download structures
for pdb in pdb_structures:
    fetch_updated_mmcif(
        pdb_code=pdb,
        path_save=path_save_ummcifs
    )


#### 3.2) Create clustering object

All structures to be clustered are now available locally. Next, identify the chains to be parsed into the clustering object. `ClusterConformations()` reads the `label_asym_id` loop (sometimes called `struct_asym_id`) chain ID, so parsing in the author-specified (`auth_asym_id`) will return an error. 

`ClusterConformations()` reads in the structures to be clustered as a dictionary in the following format:

```python
input_dict = {
        "/path/to/updated/mmcif/1atp_updated.cif" : ['A', 'B'],
        "/path/to/updated/mmcif/2adp_updated.cif" : ['C', 'D', 'E'],
        ...
        "/path/to/updated/mmcif/9amp_updated.cif" : ['A', 'B', ... 'Z']
}
```

The function `cluster_conformers.find_conformers.extract_structure_format` can be used to parse command-line input into this dictionary format. An input dictionary is prepared below before the clustering object `conformations` is initialised. 

In [ ]:

from cluster_conformers.cluster_monomers import ClusterConformations

# Specify your chains for clustering
input_dictionary = {
    "updated_mmcifs/P46060/5d2m_updated.cif" : ['C', 'F'],
    "updated_mmcifs/P46060/2grn_updated.cif" : ['B'],
    "updated_mmcifs/P46060/2grp_updated.cif" : ['B'],
    "updated_mmcifs/P46060/2grq_updated.cif" : ['B'],
    "updated_mmcifs/P46060/2gro_updated.cif" : ['B'],
    "updated_mmcifs/P46060/2grr_updated.cif" : ['B'],
    "updated_mmcifs/P46060/3uip_updated.cif" : ['C'],
    "updated_mmcifs/P46060/1z5s_updated.cif" : ['C'],
    "updated_mmcifs/P46060/3uin_updated.cif" : ['C'],
    "updated_mmcifs/P46060/2iy0_updated.cif" : ['C'],
    "updated_mmcifs/P46060/3uio_updated.cif" : ['C'],
    "updated_mmcifs/P46060/2io3_updated.cif" : ['C'],
    "updated_mmcifs/P46060/2io2_updated.cif" : ['C']
}

# Create clustering object
conformations = ClusterConformations(
    unp="P46060",
    mmcifs_and_chains=input_dictionary
)

By default, the object will run all future methods with 10 process (if available) and skip generation of matrices for extant files. To change these default, use the `nproc` and `force` arguments, respectively, where the latter regenerates all matrices, regardless of their existance. 

```python
unp_cluster = cluster_monomers.ClusterConformations(
        unp="A12345",
        mmcifs_and_chains={
            "/path/to/updated_mmcif.cif" : ['A', 'B'],
            ...
        },
        nproc=10,
        force=True,
    )
```

For most modern PCs, we find 10-cores is optimal for minimising runtime, although you may want to adjust this parameter to suit your hardware. 

 <img src="figures/multiprocessed_performance_benchmarked.png" width="1000"/>
 
 Specifications:
 - CPU: AMD Ryzen 9 5900X (24) @ 4.175GHz
 - Memory: 3501MiB / 32019MiB
 - Storage: 1TB Samsung M.2 NVMe 980PRO (SSD)
 - OS: Linux Mint 21 x86_64

Once the `conformations` object has been created, call the `.ca_distances()` method to generate all-to-all backbone C-alpha Euclidean distances between all observed residues in your set of parsed structures. Pairwise CA distances between each residue and every other residue must next be generated. 

In [ ]:
# Generate CA distance matrices
conformations.ca_distance(
    path_save_ca_matx="ca_distance_matrices"
)

The path to save the C-alpha distance matrices is saved by the object as `self.path_save_ca_matx`. Therefore, do not remove C-alpha matrices from the specified directory, unless this attribute is updated. 

C-alpha distance difference matrices must be generated before the clustering method `.cluster()` can be called. `.cluster()` requires two arguments: `path_save_dd_matx` and `path_save_cluster_results` to run. Although you may not require distance-difference matrices for later use, they are stored in order to reduce the program's memory overhead, which can be very large when many chains are parsed. An `N*N` matrix of `CA_ij - CA_ij` distances, where `N` represents the chain lengths and `i,j <= N`. These matrices are then saved to your hard drive and only a reference to it is saved in memory. To save space, the matrices are compressed to `.npz` serialised objects, floats converted to `float32` (it is unlikely any CA-CA distance exceeds the 32-bit limit) and elements below 3 Angstroms are set to 0.

In [ ]:
# Perform clustering and save results
conformations.cluster(
    path_save_dd_matx="distance_difference_matrices",
    path_save_cluster_results="cluster_results"
)

For an example of how to use `ClusterConformations()`, see `find_conformers.py` in the root of the repository. 

##### 3.2.1) Including AlphaFold structures in clustering

AlphaFold structures for the given UniProt can also be included in the clustering process. To do this, simply specify the `path_save_alphafold` argument when initialising a `ClusterConformations()` object. You do not need to install the AlpgaFold structure _a priori_, providing you have an internet connection. 

##### 3.2.2) Updating select chains

To update only a selection of chains in the original set of parsed structures, use the `remove_entry_matxs()` method before calling `ca_distance()` and leave `force=False` (default). This method will remove the C-alpha distance matrices and distance-difference matrices for the specified chains, allowing them to be regenerated when `ca_distance()` and `cluster()` are called. For example:

In [ ]:
"""
Generate cluster object and perform clustering for set of chains, where "1atp" chain A has been changed. 
"""

# Initialise cluster object with all chains for clustering
conformers = ClusterConformations(
    unp="A12345",
    mmcifs_and_chains={
        "updated_mmcifs/A12345/1atp_updated.cif" : ['A'],
        "updated_mmcifs/A12345/1adp_updated.cif" : ['B'],
        "updated_mmcifs/A12345/1amp_updated.cif" : ['B'],
    },
    nproc=1,
    force=False
)

# Remove existing matrices for updated entries -- they will be regenerated later
conformations.remove_entry_matxs(
    pdb_ids={"1atp"},       # Should be parsed as set
    path_ca="/path/to/ca_distance_matrices",
    path_dd="/path/to/distance_difference_matrices",
)

# Generate CA distance matrices -- only for updated entries, which were removed above
conformations.ca_distance(
    path_save_ca_matx="path/to/ca_distance_matrices"
)

# Perform clustering and save results
conformations.cluster(
    path_save_dd_matx="path/to/distance_difference_matrices",
    path_save_cluster_results="path/to/cluster_results"
)

----

## 3) Interpretting results

Once clustering for the set of structures has been completed, the results can be visualised using the `render_dendrogram()` function. It requires the `path_save_cluster_results` argument, which should point to the same location as `path_save_cluster_results` when `.cluster()` was called. 

In [ ]:
from cluster_conformers.cluster_monomers import render_dendrogram

# Render dendrogram 
render_dendrogram(
    unp="A12345",
    # Generated by ClusterConformations.cluster()
    path_results="path/to/cluster_results",     
    # Path to save dendrogram
    path_save="/path/to/dendrogram",
    png=True,       # Save as png
    svg=False,      # (and/or) Save as svg
)

 <img src="figures/kaib_segment1_dendogram.png" width="500"/>

Example of PNG dendrogram rendered with the `render_dendrogram()` function. 

You may also find it useful to visualise the distance-difference matrices generated from the clustering process. You can call the `make_dd_maps()` function to generate a series of heatmap PNGs. The function is called by the `find_conformers()` but its general usage is as follows:


In [ ]:
from cluster_conformers.distance_differences import make_dd_maps

make_dd_maps(
    # Should be same as used in ClusterConformations.cluster()
    path_matxs="path/to/distance_difference_matrices",
    path_save="path/to/dd_maps",
    force=False
)

In order to work, the ClusterConformations.cluster() method must have been called with the `path_save_dd_matx` argument specified as `path_matxs` here. As many PNG images as matrices will be generated, so directories with many matrices could be time-consuming to render. The `force` argument can be used to regenerate all PNGs, regardless of their existance. 

Here is an example of a rendered distance-difference matrix:

 <img src="figures/A6UVT1_example_dd_map.png" width="500"/>

Matrices will all be normalised their maximum value across all matrices in the parsed dircetory. Currently, this behaviour cannot be toggle, but it is often inconvinient to visualise matrices with different scales.

----

## 4) Cluster conformers in benchmark dataset

Prior to September 2022, the PDBe-KB's clustering pipeline (of which the worker function used is described here) scored conformational differences based on the Q-score (a normalisation of RMSD) returned by the structural aligner GESAMT. Q-score is ordinarily used for scoring the superposition of parsed peptide chains, which are fragmented to obtain good alignment around structurally conserved regions. 

However, it was apparent high Q-scores (although leading to excellent structural alignment in many cases) would often be obtained for only the immobile regions of otherwise conformationally heterogenous peptides. This requently lead to distinct conformational states getting grouped together. 

Therefore, we developed a benchmark dataset (available on the PDBe's [FTP server](http://ftp.ebi.ac.uk/pub/databases/pdbe-kb/benchmarking/distinct-monomer-conformers/) and [Kaggle page](https://www.kaggle.com/datasets/josephellaway/distinct-monomeric-protein-conformers)) to test whether separating superposition from conformational state recognition (detailed [here](link_to_supp_information)) would mitigate this limitation. We found that it massively reduces the number of conformational states from a maxumum of 1,600 to only 9. 

The figure below graphcially summarises the contents of this benchmark dataset.

 <img src="figures/monomer_benchmark_ds_summary_info.png" width="700"/>

Some additional tools have been provided to facilitate easy access to the PDBe-KB's clustering data. 


